In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing	import OneHotEncoder
from datetime import datetime
import pickle
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.tree import DecisionTreeClassifier

load_dotenv()
os.chdir(os.getenv("ROOT_DIR"))

from src import utils

/home/gonzalo/Documents/notes/p-ds-structure-01/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- PARAMS --- #
prep_id = 'prep_01'


In [3]:
# --- LOAD --- #
x_train, y_train, x_test, y_test = utils.get_prep_df(prep_id)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(712, 3) (712, 1) (179, 3) (179, 1)


In [4]:
############################################
### DECISION-TREE-GDOC
### sklearn.tree.DecisionTreeClassifier
### sklearn v.1.3.2.
############################################

### Main ref ----------------------------------------------------------------
# - https://scikit-learn.org/stable/modules/tree.html#tree

### Main def --------------------------------
# Decision Trees (DTs) are a non-parametric supervised learning method 
# used for classification and regression. The goal is to create a model 
# that predicts the value of a target variable by learning simple decision 
# rules inferred from the data features. A tree can be seen as a 
# piecewise constant approximation.

### Main advantages --------------------------------
# - Simple to understand and to interpret. Trees can be visualized. White box model.
# - Fast: The cost of using the tree (i.e., predicting data) is logarithmic 
# in the number of data points used to train the tree.

### Main disadvantages --------------------------------
# - Decision-tree learners can create over-complex trees that do not 
# generalize the data well (overfitting).
# - Decision trees can be unstable because small variations in the 
# data might result in a completely different tree being generated. 
# This problem is mitigated by using decision trees within an ensemble.
# - The problem of learning an optimal decision tree is known to be NP-complete 
# under several aspects of optimality and even for simple concepts. 
# Consequently, practical decision-tree learning algorithms are based on 
# heuristic algorithms such as the greedy algorithm where locally optimal 
# decisions are made at each node. Such algorithms cannot guarantee to 
# return the globally optimal decision tree. This can be mitigated by 
# training multiple trees in an ensemble learner, where the features 
# and samples are randomly sampled with replacement.
# - Decision tree learners create biased trees if some classes dominate. 
# It is therefore recommended to BALANCE the dataset prior 
# to fitting with the decision tree.

### Implementation Highlights -------------------------------------------------
# - DecisionTreeClassifier is a class capable of performing multi-class 
# classification on a dataset
# - the scikit-learn implementation does not support categorical variables.

### Signature ----------------------------------------------------------------
# ref: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

#DecisionTreeClassifier(
# criterion='gini', # metric to define que quality of a split.
# splitter='best', # The best possible split is used or we use a random split?
# max_depth=None, #!key to control overfitting
# min_samples_split=2, 
# min_samples_leaf=1, #!key to create asymmetric trees. 
# min_weight_fraction_leaf=0.0, 
# max_features=None, # how many features consider when lookint for the best split. See doc.
# random_state=None, 
# max_leaf_nodes=None, 
# min_impurity_decrease=0.0, 
# class_weight=None, #!  dict, list of dict or “balanced”, default=None
# ccp_alpha=0.0
#)

#DecisionTreeClassifier(
# criterion='gini',
# splitter='best', 
# max_depth=None, 
# min_samples_split=2, 
# min_samples_leaf=1, 
# min_weight_fraction_leaf=0.0, 
# max_features=None, 
# random_state=None, 
# max_leaf_nodes=None, 
# min_impurity_decrease=0.0, 
# class_weight=None, 
# ccp_alpha=0.0
#)

### Hyperparams Comments  --------------------------------
# - The max_depth hyperparameter controls the overall complexity of the tree. 
# This parameter is adequate under the assumption that a tree is built symmetrically. 
# However, there is no reason why a tree should be symmetrical. Indeed, optimal generalization 
# performance could be reached by growing some of the branches deeper than some others.

# The hyperparameters min_samples_leaf, min_samples_split, 
# max_leaf_nodes, or min_impurity_decrease allow growing asymmetric trees and apply 
# a constraint at the leaves or nodes level. 
# ref. https://inria.github.io/scikit-learn-mooc/python_scripts/trees_hyperparameters.html

In [5]:
# --- TUNNING ---
# --- Objective function: define the optimization metrics ---
# (where, for what model, what metric)

def objective(trial, x, y):
	
				# define search space
				params = {
								'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
								'splitter': trial.suggest_categorical('splitter', ['best', 'random']),
								'max_depth': trial.suggest_int('max_depth', 1, 10),
								'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
								'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32),
								'random_state': 0
				}

				# define model
				clf	= DecisionTreeClassifier(**params)

				# Eval strategy
				# It gives me the list of k scores from cv
				cv_scores = cross_val_score(clf, x, y, cv=3, scoring='accuracy')

				obj = cv_scores.mean()
				
				# return score for each trial
				return obj

w_func = lambda trial: objective(trial, x=x_train, y=y_train) # just a wrapper	to pass x and y

# --- study: define searching strategy (How) ---

# sampler=TPESampler(): Optuna uses the Tree-structured Parzen Estimator (TPE) 
# sampler, which is a Bayesian optimization method that efficiently searches 
# through the hyperparameter space.

# pruner=MedianPruner(): Optuna can prune unpromising trials early, based on the median 
# value of intermediate results. This speeds up the optimization process by discarding 
# poor-performing hyperparameter combinations early.

study=optuna.create_study(
	sampler= optuna.samplers.TPESampler(),
	pruner=optuna.pruners.MedianPruner(),
	direction='maximize'
	)

study.set_metric_names(["accuracy"])


# -- Start optimizing ---
study.optimize(
	func=w_func, 
	n_trials=100,
	timeout=None, # max time in seconds
	n_jobs=-1 # max job  in parallel. -1 = all cpus
	)

[I 2024-10-10 19:37:59,964] A new study created in memory with name: no-name-9e7de40f-e242-4d3d-ba94-5f53029ed337
/tmp/ipykernel_1134114/1392542347.py:47: ExperimentalWarning: set_metric_names is experimental (supported from v3.2.0). The interface can change in the future.
  study.set_metric_names(["accuracy"])
[I 2024-10-10 19:38:00,127] Trial 1 finished with value: {'accuracy': 0.7696758028105757} and parameters: {'criterion': 'gini', 'splitter': 'random', 'max_depth': 8, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.7696758028105757.
[I 2024-10-10 19:38:00,136] Trial 0 finished with value: {'accuracy': 0.7696758028105757} and parameters: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.7696758028105757.
[I 2024-10-10 19:38:00,157] Trial 4 finished with value: {'accuracy': 0.7865534399413775} and parameters: {'criterion': 'gini', 'splitter': 'best', 'max_depth'

In [7]:
# save tunning results
utils.tunning_results(study, os.getenv("ARTIFACTS_PATH"))

Results saved in artifacts/model_01/tunning


{'best_params': {'criterion': 'gini',
  'splitter': 'best',
  'max_depth': 1,
  'min_samples_split': 10,
  'min_samples_leaf': 12},
 'best_value': 0.7865534399413775,
 'metric_names': ['accuracy']}

In [8]:
# --- TRAIN ---
x_full = pd.concat([x_train, x_test])
y_full = pd.concat([y_train, y_test])

best_params = study.best_params
clf = DecisionTreeClassifier(**best_params)
model = clf.fit(x_full, y_full)

# --- save as pickle ---
artifact_path = os.path.join(os.getenv("ARTIFACTS_PATH"), utils.get_nb_name(), 'model')
os.makedirs(artifact_path, exist_ok=True)

with open(os.path.join(artifact_path, 'model.pkl'), 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved at: {os.path.join(artifact_path, 'model.pkl')}")
print(f'Timestamp: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')


Model saved at: artifacts/model_01/model/model.pkl
Timestamp: 2024-10-10 19:38:06
